# Call Context Summary

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy

import anytree

import paptree

## 2. Load Data

In [2]:
trace_file = pathlib.Path.cwd().parent / "data/fibonacci/paptrace.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Loaded 36 traces.


In [3]:
# Take a peek at a few of the traces.
for tree in trees[:3]:
    print(anytree.RenderTree(tree))
    print()

CallNode(name=2106190, params=[{'name': 'n', 'value': '0'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2106009, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2106007, type='ReturnStmt')

CallNode(name=2106190, params=[{'name': 'n', 'value': '1'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2106009, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2106007, type='ReturnStmt')

CallNode(name=2106190, params=[{'name': 'n', 'value': '2'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)', name=2106188, type='ReturnStmt')
    └── StmtNode(desc='unsigned long long', name=2106184, type='+')
        └── StmtNode(desc='int', name=2106117, type='-')
            ├── CallNode(name=2106190, params=[{'name': 'n', 'value'

Each trace entry from `paptrace` is a walk of an execution path walk for a function and a specific set of input parameters. The first trace is a walk of the function `fibonacci::RecursiveNaive` for `n = 0`.

In [4]:
for tree in trees[18:27]:
    print(anytree.RenderTree(tree))

CallNode(name=2110045, params=[{'name': 'n', 'value': '0'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2109766, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2109764, type='ReturnStmt')
CallNode(name=2110045, params=[{'name': 'n', 'value': '1'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2109766, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2109764, type='ReturnStmt')
CallNode(name=2110045, params=[{'name': 'n', 'value': '2'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
├── StmtNode(desc='for (unsigned short i = 2; i <= n; ++i)', name=2110029, type='ForStmt')
│   ├── StmtNode(desc='LoopIter', name=2110029, type='LoopIter')
│   │   ├── StmtNode(desc='unsigned long long', name=2109990, type='=')
│   │   │   └── StmtNode(desc='unsigned long long', name=2109986, type='+')
│   │   ├── S

## 3. Extract Call Nodes

In [5]:
call_nodes = []
for tree in trees:
    call_nodes.extend(
        anytree.findall(tree.root, filter_=lambda n: isinstance(n, paptree.CallNode)))

print(f"Number of call nodes: {len(call_nodes)}")

Number of call nodes: 288


In [6]:
sigs = set([node.sig for node in call_nodes])
print(f"Number of unique signatures: {len(sigs)}")
display(sigs)

Number of unique signatures: 6


{'reference std::vector<unsigned long long>::operator[](size_type)',
 'unsigned long long fibonacci::(anonymous namespace)::RecursiveMemoImpl(unsigned short, std::vector<unsigned long long> &)',
 'unsigned long long fibonacci::Iterative(unsigned short)',
 'unsigned long long fibonacci::LookupTable(unsigned short)',
 'unsigned long long fibonacci::RecursiveMemo(unsigned short)',
 'unsigned long long fibonacci::RecursiveNaive(unsigned short)'}

Of the 288 call nodes, only 6 unique signatures were detected. How many contexts are there per signature? How many unique contexts per signature?

In [7]:
print(f"Unique call nodes: {len(set([str(n) for n in call_nodes]))}")

Unique call nodes: 184


## 4. Summarize Call Contexts

In [8]:
call_map = {}
for call_node in unique_call_nodes:
    id = call_node["id"]
    if not id in call_map:
        call_map[id] = {"sig": call_node["sig"], "ctxs": {}}
    ctx = str(call_node["params"])
    if not ctx in call_map[id]["ctxs"]:
        call_map[id]["ctxs"][ctx] = len(call_map[id]["ctxs"])
    else:
        raise Exception(f"Duplicate call node ctx: {ctx}!")
        
print(f"Number of top-level call map entries: {len(call_map)}")
display(call_map)

NameError: name 'unique_call_nodes' is not defined